# pivot_utms

In [2]:
library(readxl)
library(dplyr)
library(tidyr)
library(janitor)   
library(stringr)

df <- read_excel(
  '/Users/eugenetsenter/Downloads/MASSMUTUAL003CP_Q1_Trafficking Sheet_DSP_CTV, OLV, & Audio ONLY_3.17.25 (2).xlsx',
  sheet = "Q1 Tsheet",
  skip = 6,
) %>% 
  clean_names()                          # camelCase → snake_case



# Make all creative columns follow creative_<number>_<attribute>
names_test <- str_remove(str_replace(str_replace(names(df),"creative_",""),"number_",""),"\\d*") 
names_prefix <- paste0("creative_",str_extract(names(df),"-?\\d*\\.?\\d+"),"__", names_test)
names(df) <- names_prefix


names(df) <- names(df) |>
  # Remove trailing numbers from names: creative_1__name_1 → creative_1__name
  str_replace("(_name|_url|_asset_link|_edo_tag|_disqo_tag|_video_amp_tag|_3p_or_1p_tag)_\\d+$", "\\1") |>
  # Remove all sequences of 2 or more underscores: creative_1__name → creative_1_name
  str_replace_all("_+", "_") |>
  # Fix creative_3__url_3_number → creative_3_url
  str_replace("_url_3_number", "_url")


# 2️⃣  Identify all creative columns ---------------------------------------
cre_cols <- grep("(name|url|asset_link|edo_tag|disqo_tag|video_amp_tag|3p_or_1p_tag)",
                 names(df), value = TRUE)

library(tidyr)
creative_long <- df %>%
  pivot_longer(
    cols = matches("^creative_\\d+_"),
    names_to = c("creative_num", ".value"),
    names_pattern = "^creative_(\\d+)_(.*)$",
    values_drop_na = TRUE
  ) %>%
  mutate(creative_num = as.integer(creative_num))
# 



Attaching package: 'dplyr'


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union


Warning message:
"package 'janitor' was built under R version 4.3.3"

Attaching package: 'janitor'


The following objects are masked from 'package:stats':

    chisq.test, fisher.test




# write to table

In [5]:
# write to table
library(DBI)
library(bigrquery)

con <- dbConnect(
    bigrquery::bigquery(),
    project = "looker-studio-pro-452620",
    dataset = "20250327_data_model",
    billing = "looker-studio-pro-452620"
)

dbWriteTable(
    con,
    name = "basis_utms_pivoted",
    value = df,
    overwrite = TRUE
)

In [7]:
head(df) %>% print(n = 20)

# A tibble: 6 x 103
  creative_NA_property creative_NA_line_item              creative_NA_tag_plac~1
  <chr>                <chr>                              <chr>                 
1 Basis DSP            NA                                 NA                    
2 Basis  DSP           MASSMUTUAL003CP_757075_Q1_Mid Fun~ MASSMUTUAL003CP_31271~
3 Basis  DSP           MASSMUTUAL003CP_757075_Q1_Mid Fun~ MASSMUTUAL003CP_31271~
4 Basis  DSP           MASSMUTUAL003CP_757075_Q1_Mid Fun~ MASSMUTUAL003CP_31271~
5 Basis  DSP           MASSMUTUAL003CP_757075_Q1_Mid Fun~ MASSMUTUAL003CP_31271~
6 Basis  DSP           MASSMUTUAL003CP_757075_Q1_Mid Fun~ MASSMUTUAL003CP_31271~
# i abbreviated name: 1: creative_NA_tag_placement
# i 100 more variables: creative_NA_formats <chr>, creative_NA_size <chr>,
#   creative_NA_start_date <dttm>, creative_NA_end_date <dttm>,
#   creative_1_name <chr>, creative_1_url <chr>, creative_1_rotation_1 <dbl>,
#   creative_1_asset_link <chr>, creative_1_edo_tag <chr>,
#   c